# Rekognition Custom Labels の APIレスポンスを分析して、最適な閾値を探す方法

Amazon Rekognition Custom Labelsのモデルができても、結果の評価が難しいと感じたことはありませんか？Assumed Thresholdsをカスタマイズして、モデルの評価方法を見直す方法をご紹介します。

### モデルの開始

モデルを推論エンドポイントとしてデプロイします。課金が分単位で発生 ($4/hr, 2021/12/01現在) するので注意してください。

In [ ]:
#Copyright 2020 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-custom-labels-developer-guide/blob/master/LICENSE-SAMPLECODE.)

import boto3
from typing import Dict

def start_model(project_arn: str, model_arn: str, version_name: str, min_inference_units: int) -> None:

    client=boto3.client('rekognition')

    try:
        # Start the model
        print('Starting model: ' + model_arn)
        response=client.start_project_version(ProjectVersionArn=model_arn, MinInferenceUnits=min_inference_units)
        # Wait for the model to be in the running state
        project_version_running_waiter = client.get_waiter('project_version_running')
        project_version_running_waiter.wait(ProjectArn=project_arn, VersionNames=[version_name])

        #Get the running status
        describe_response=client.describe_project_versions(ProjectArn=project_arn,
            VersionNames=[version_name])
        for model in describe_response['ProjectVersionDescriptions']:
            print("Status: " + model['Status'])
            print("Message: " + model['StatusMessage']) 
    except Exception as e:
        print(e)
        
    print('Done...')
    
def get_arns(model_arn: str) -> Dict[str, str]:
    rek_arn = model_arn.split('/')[0]
    project_num = model_arn.split('/')[-1]
    project_name = model_arn.split('/')[-4]
    project_arn = f"{rek_arn}/{project_name}/{project_num}"
    version_name = model_arn.split('/')[-2]
    ans = {}
    ans['project_arn'] = project_arn
    ans['version_name'] = version_name
    return ans

In [ ]:
# ここにRekognition Custom Labelsの学習済みモデルのARNをコピペしてください
MODEL_ARN = ""
min_inference_units = 1
arn_info = get_arns(MODEL_ARN)
project_arn = arn_info['project_arn']
version_name = arn_info['version_name']
start_model(project_arn, MODEL_ARN, version_name, min_inference_units)

### モデルの使用

In [ ]:
#Copyright 2020 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-custom-labels-developer-guide/blob/master/LICENSE-SAMPLECODE.)

import boto3
import io
from PIL import Image, ImageDraw, ExifTags, ImageColor, ImageFont

def display_image(bucket: str, photo: str, response: Dict[str, str]) -> None:
    # Load image from S3 bucket
    s3_connection = boto3.resource('s3')

    s3_object = s3_connection.Object(bucket,photo)
    s3_response = s3_object.get()

    stream = io.BytesIO(s3_response['Body'].read())
    image=Image.open(stream)

    # Ready image to draw bounding boxes on it.
    imgWidth, imgHeight = image.size
    draw = ImageDraw.Draw(image)

    # calculate and display bounding boxes for each detected custom label
    print('Detected custom labels for ' + photo)
    for customLabel in response['CustomLabels']:
        print('Label ' + str(customLabel['Name']))
        print('Confidence ' + str(customLabel['Confidence']))
        if 'Geometry' in customLabel:
            box = customLabel['Geometry']['BoundingBox']
            left = imgWidth * box['Left']
            top = imgHeight * box['Top']
            width = imgWidth * box['Width']
            height = imgHeight * box['Height']

            fnt = ImageFont.truetype('/Library/Fonts/Arial.ttf', 50)
            draw.text((left,top), customLabel['Name'], fill='#00d400', font=fnt)

            print('Left: ' + '{0:.0f}'.format(left))
            print('Top: ' + '{0:.0f}'.format(top))
            print('Label Width: ' + "{0:.0f}".format(width))
            print('Label Height: ' + "{0:.0f}".format(height))

            points = (
                (left,top),
                (left + width, top),
                (left + width, top + height),
                (left , top + height),
                (left, top))
            draw.line(points, fill='#00d400', width=5)

    image.show()

def show_custom_labels(model: str, bucket: str, photo: str, min_confidence: int) -> Dict[str, str]:
    client=boto3.client('rekognition')

    # Call DetectCustomLabels
    response = client.detect_custom_labels(Image={'S3Object': {'Bucket': bucket, 'Name': photo}},
        MinConfidence=min_confidence,
        ProjectVersionArn=model)

    # For object detection use case, uncomment below code to display image.
    display_image(bucket,photo,response)
    
    # Add an image info for each dict
    for d in response['CustomLabels']:
        d['image'] = photo
        
    return response['CustomLabels']

In [ ]:
# テスト画像のURIとラベルを取得
import json
import os
tests = []

# この.ipynbファイルと同じフォルダに'testing_manifest_with_validation.json'があると仮定します。適宜変更してください。
with open('testing_manifest_with_validation.json', 'r') as f:
    for line in f:
        tests.append(json.loads(line))

In [ ]:
tests

In [ ]:
estimations = []

for i in range(len(tests)):
    # データセットが格納されているS3バケット名を指定してください
    BUCKET_NAME = ""
    photo = '/'.join(tests[i]['source-ref'].split('/')[-2:])
    min_confidence=0
    
    estimations.append(show_custom_labels(MODEL_ARN, BUCKET_NAME, photo, min_confidence))
    
print(estimations)

In [ ]:
estimations[0]

In [ ]:
# 実運用でも、こちらの関数を使えます。
def filter_results(results, thresholds, verbose=1):
    '''
    results: JSON responses from Rekognition Custom Labels
    thresholds: { class_name: string, confidence: float }
    verbose: Prints out the progress on stdout
    Return filtered responses
    '''
    '''
    RekognitionのAPIレスポンスとあらかじめ定義されたthresholdsから
    以下のロジックに従ってあてはまるラベルを１つに定める関数
    1. それぞれのラベルに対して、信頼度がthresholdを超えるラベルをマークする
    2. どのラベルもthresholdを超えない場合は、専門家の判断を仰ぐ
    3. 複数ラベルが該当する場合は、最も信頼度が高いラベルを選ぶ
    4. １つのラベルのみが該当する場合は、そのラベルを選ぶ
    '''
    # １つ目のAPIレスポンスからすべてのラベルクラスを抽出
    classes = list(map(lambda x: x['Name'], results[0]))
    # ここに予測に当てはまるラベルを１つに決めて貯める。信頼度が低すぎて人の目を通す必要になるものも含む。
    labeling_results = []
    for result in results:
        image_info = result[0]['image']
        # このレスポンスに対して、信頼度がthresholdを超えていればflagsでTrueにセットする
        flags = {key: False for key in classes}
        for label in classes:
            # このループは、ラベルを特定するため
            for d in result:
                if d['Name'] == label:
                    if d['Confidence'] > thresholds[label]:
                        flags[label] = True
        # flagsからTrueになったラベルの数を数える
        num_true = sum(flags.values())
        if num_true == 0:
            # 与えられたThresholdではモデルが判定できない
            if verbose >= 1:
                print(f"この画像はうまく判断できません！専門家の判断を仰いでください: {image_info}")
            final_result = {'Name': 'human', 'image': image_info}
        elif num_true > 1:
            # 複数のラベルが付いている
            labels = list(filter(lambda key: flags[key], flags.keys()))
            filtered_result = list(filter(lambda x: x['Name'] in labels, result))
            final_result = max(filtered_result, key = lambda d: d['Confidence'])
        else:
            # ラベルが１つだけに定まっている
            final_label = list(filter(lambda key: flags[key], flags.keys()))[0]
            final_result = list(filter(lambda d: d['Name'] == final_label, result))[0]
        labeling_results.append(final_result)
        if verbose >= 1:
            print(f"Raw: {result}\nLabel: {final_result}\n\n")
    return labeling_results

In [ ]:
thresholds = {'Class A': 50, 'Class B': 50, 'Class C': 50}
filter_results(estimations, thresholds)

## 様々なThresholdsを試して人間が判断する枚数とRekognitionが振り分けた画像のPrecisionのバランスを取る

なるべく人間が判断する枚数を減らしつつ、それぞれのクラスに対してのPrecisionが許容できる範囲内に収める

In [ ]:
# このセルはループが100万回ほど回るため、ある程度時間がかかります。
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import precision_recall_fscore_support

# type [{'precisions': float[], 'thresh': Treshold, 'human_ratio': float}]
analyses = []
for a_thresh in range(0, 101):
    print(f"=== {a_thresh}% ===")
    for b_thresh in range(0, 101):
        for c_thresh in range(0, 101):
            thresh = {'Class A': a_thresh, 'Class B': b_thresh, 'Class C': c_thresh}
            labeling_results = filter_results(estimations, thresh, verbose=0)
            humans = 0
            y_pred = []
            y_true = []
            for label_obj in labeling_results:
                if label_obj['Name'] == "human":
                    humans += 1
                else:
                    y_pred.append(label_obj['Name'])
                    y_true.append(label_obj['image'].split('/')[0])
            human_ratio = humans / len(labeling_results)
            precisions, _, _, _ = precision_recall_fscore_support(y_true, y_pred)
            analyses.append({'precisions': precisions, 'thresh': thresh, 'human_ratio': human_ratio})

In [ ]:
# 人間が一切介入しないようにしたらどうなるか
analyses[0]

In [ ]:
# フィルタリングの基準を定める関数
def criteria(d: Dict, least_precision: float, most_humans: float) -> bool:
    return len(d['precisions']) > 0 and min(d['precisions']) >= least_precision and d['human_ratio'] <= most_humans

def explore_analysis(least_precision: float, most_humans: float):
    from pprint import pprint
    result = list(filter(lambda d: criteria(d, least_precision, most_humans), analyses))
    print(f"Precisionが最低でも{least_precision*100}%で人間が判断するのが最大でも{most_humans*100}%になるようなThresholdは{len(result)}個みつかりました。\n")
    
    if len(result) > 0:
        best_result = [
            # この中でprecisionsの最小値が最大のもの
            max(result, key=lambda d: min(d['precisions'])),
            # この中で人間への負荷が最小のもの
            min(result, key=lambda d: d['human_ratio'])
        ]
        print("この範囲ならこの２つがベストです↓\n")
        pprint(best_result)
    
    return result

In [ ]:
# 例１: Precisionが最低でも95%で人間が判断するのが最大でも50%になるようなThresholdを探したい！
result1 = explore_analysis(0.95, 0.5)

In [ ]:
# 例２: Precisionが最低でも85%で人間が判断するのが最大でも25%になるようなThresholdを探したい！
result2 = explore_analysis(0.85, 0.25)

In [ ]:
# 例３: Precisionが最低でも95%で人間が判断するのが最大でも10%になるようなThresholdを探したい！
result3 = explore_analysis(0.8, 0.2)

### モデルの停止

In [ ]:
#Copyright 2020 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-custom-labels-developer-guide/blob/master/LICENSE-SAMPLECODE.)

import boto3
import time


def stop_model(model_arn):

    client=boto3.client('rekognition')

    print('Stopping model:' + model_arn)

    #Stop the model
    try:
        response=client.stop_project_version(ProjectVersionArn=model_arn)
        status=response['Status']
        print ('Status: ' + status)
    except Exception as e:  
        print(e)  

    print('Done...')

In [ ]:
stop_model(model_arn)